## 1. PLSC-SwinTransformer Introduction


PLSC-SwinTransformer reimplementation of [microsoft's repository for the Swin-Transformer](https://github.com/microsoft/Swin-Transformer) model that was released with the paper [Swin Transformer: Hierarchical Vision Transformer using Shifted Windows](https://arxiv.org/pdf/2103.14030.pdf).

Swin Transformer (the name Swin stands for Shifted window) capably serves as a general-purpose backbone for computer vision. It is basically a hierarchical Transformer whose representation is computed with shifted windows. The shifted windowing scheme brings greater efficiency by limiting self-attention computation to non-overlapping local windows while also allowing for cross-window connection.

![Figure 1 from paper](https://github.com/microsoft/Swin-Transformer/blob/main/figures/teaser.png?raw=true)


## 2. Model Effects

| Model |DType | Phase | Dataset | gpu | img/sec | Top1 Acc | Official |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Swin-B |FP16 O1|pretrain  |ImageNet2012  |A100*N1C8  |  2155| 0.83362 | 0.835 |
| Swin-B |FP16 O2|pretrain  | ImageNet2012 | A100*N1C8 | 3006 | 0.83223	 | 0.835 |


## 3. How to use the Model

### 3.1 Install PLSC

```
git clone https://github.com/PaddlePaddle/PLSC.git
cd /path/to/PLSC/
# [optional] pip install -r requirements.txt
python setup.py develop
```

### 3.2 Model Training

1. Enter into the task directory

```
cd task/classification/swin
```

2. Prepare the data

Organize the data into the following format:


```text
dataset/
└── ILSVRC2012
    ├── train
    ├── val
    ├── train_list.txt
    └── val_list.txt
```

3. Run the command

```shell
export PADDLE_NNODES=1
export PADDLE_MASTER="127.0.0.1:12538"
export CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7

python -m paddle.distributed.launch \
    --nnodes=$PADDLE_NNODES \
    --master=$PADDLE_MASTER \
    --devices=$CUDA_VISIBLE_DEVICES \
    plsc-train \
    -c ./configs/swin_base_patch4_window7_224_in1k_1n8c_dp_fp16o1.yaml
```

More courses about model training can be learned here [Swin](https://github.com/PaddlePaddle/PLSC/blob/master/task/classification/swin/README.md)

### 3.3 Model Inference

1. Download pretrained model and image


```shell
# download pretrained model
mkdir -p pretrained/swin/Swin_base/
wget -O ./pretrained/swin/Swin_base/swin_base_patch4_window7_224_fp16o1.pdparams 
https://plsc.bj.bcebos.com/models/swin/v2.5/swin_base_patch4_window7_224_fp16o1.pdparams

# download image
mkdir -p images/
wget -O ./images/zebra.png https://plsc.bj.bcebos.com/dataset/test_images/zebra.png
```



2. Export model for inference

```shell
plsc-export -c ./configs/swin_base_patch4_window7_224_in1k_1n8c_dp_fp16o1.yaml -o Global.pretrained_model=./pretrained/swin/Swin_base/swin_base_patch4_window7_224_fp16o1 -o Model.data_format=NCHW -o FP16.level=O0
```


3. Image inference

In [ ]:
import numpy as np

from plsc.data.dataset import default_loader
from plsc.data.preprocess import Resize
from plsc.engine.inference import Predictor


def preprocess(img):
    resize = Resize(size=224, 
                    interpolation="bicubic", 
                    backend="pil")
    img = np.array(resize(img))
    scale = 1.0 / 255.0
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = (img * scale - mean) / std
    img = img[np.newaxis, :, :, :]
    img = img.transpose((0, 3, 1, 2))
    return {'x': img.astype('float32')}


def postprocess(logits):
    
    def softmax(x, epsilon=1e-6):
        exp_x = np.exp(x)
        sfm = (exp_x + epsilon) / (np.sum(exp_x) + epsilon)
        return sfm

    pred = np.array(logits).squeeze()
    pred = softmax(pred)
    pred_class_idx = pred.argsort()[::-1][0]
    return pred_class_idx, pred[pred_class_idx]


infer_model = "./output/swin_base_patch4_window7_224/swin_base_patch4_window7_224.pdmodel"
infer_params = "./output/swin_base_patch4_window7_224/swin_base_patch4_window7_224.pdiparams"

predictor = Predictor(
    model_file=infer_model,
    params_file=infer_params,
    preprocess_fn=preprocess,
    postprocess_fn=postprocess)

image = default_loader("./images/zebra.png")
pred_class_idx, pred_score = predictor.predict(image)

## 4. Related papers and citations

```text
@inproceedings{liu2021Swin,
  title={Swin Transformer: Hierarchical Vision Transformer using Shifted Windows},
  author={Liu, Ze and Lin, Yutong and Cao, Yue and Hu, Han and Wei, Yixuan and Zhang, Zheng and Lin, Stephen and Guo, Baining},
  booktitle={Proceedings of the IEEE/CVF International Conference on Computer Vision (ICCV)},
  year={2021}
}
```